<a href="https://colab.research.google.com/github/UncleSamTech/301Lab1WalkThrough/blob/main/nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pickle
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import word_tokenize
import nltk
import time
import matplotlib.pyplot as plt
#import pandas as pd
import random
import scipy.stats as stats
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,confusion_matrix
import heapq
from random import sample
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_eng Averaged Perceptron Tagger (JSON)
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] averaged_perceptron_tagger_rus Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] bcp47............... BCP-47 Language Tags
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ 

       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_rus.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_d


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [3]:
scratch_model = None
ngram_model = None
loaded_scratch_model = None

In [4]:
def load_trained_model(model_name) :
        with open(model_name,"rb") as f:
            loaded_scratch_model = pickle.load(f)
        return loaded_scratch_model

In [5]:
def extract_vocabulary_nltk(model_name):
        # Load the trained MLE model from the pickle file
        model = load_trained_model(model_name)

        # Access the vocabulary (as an iterable) and convert it to a list
        vocab_list = list(model.vocab)

        return vocab_list

In [6]:
def compute_score(model_name,token,context_data):
        load_model = load_trained_model(model_name)
        token = token.strip()
        return load_model.score(token, context_data.split(" "))

In [7]:
def evaluate_mrr_nltk(model_name,result_path,split_folder,proj_number):
        # Ensure result path exists
        os.makedirs(result_path, exist_ok=True)
        all_vocab = extract_vocabulary_nltk(model_name)

        # Process each file in the split folder
        for split_file in sorted(os.listdir(split_folder)):
            split_file_path = os.path.join(split_folder, split_file)
            if not os.path.isfile(split_file_path):
                continue

            total_cumulative_rr = 0
            total_count = 0

            start_time = time.time()

            # Process each line in the file
            with open(split_file_path, "r", encoding="utf-8") as f:
                for line in f:
                    if not line.strip():
                        continue

                    # Preprocess the line
                    #line = line.replace("_", "UNDERSCORE").replace(">", "RIGHTANG").replace("<", "LEFTANG").lower()
                    sentence_tokens = line.split(" ")
                    if len(sentence_tokens) < 2:
                        continue

                    context = " ".join(sentence_tokens[:-1])
                    true_next_word = sentence_tokens[-1]

                    # Compute scores for tokens
                    heap = []
                    for token in all_vocab:
                        context_score = compute_score(model_name, token, context)
                        if len(heap) < 10:
                            heapq.heappush(heap, (context_score, token))
                        elif context_score > heap[0][0]:
                            heapq.heappushpop(heap, (context_score, token))

                    heap.sort(reverse=True, key=lambda x: x[0])
                    token_ranks = {t: rank + 1 for rank, (score, t) in enumerate(heap)}

                    # Compute reciprocal rank
                    true_next_word = true_next_word.strip()
                    rank = token_ranks.get(true_next_word, 0)
                    if rank:
                        current_rank = 1 / rank
                        total_cumulative_rr += current_rank
                        #print(f"processed line {line} with reciprocal rank {current_rank} and total cummulative {total_cumulative_rr}")
                    total_count += 1


            # Calculate total RR and lines for the file
            time_spent = time.time() - start_time
            result_file = os.path.join(result_path, f"nltk_rr_results_{proj_number}.txt")

            with open(result_file, "a") as rf:
                rf.write(f"Total Reciprocal Rank: {total_cumulative_rr}\n")
                rf.write(f"Total Lines: {total_count}\n")
                rf.write(f"Time Spent: {time_spent:.2f} seconds\n")

            print(f"Processed {split_file}: RR = {total_cumulative_rr}, Lines = {total_count}")


In [ ]:
evaluate_mrr_nltk("/content/scratch_trained_model_nltk_500_projects_6.pkl","/content/results_conf500/","/content/testfiles_split","500")
